In [15]:
import json
import numpy as np
import pandas as pd
import lxml
import re
import os
from datetime import datetime

# Script summary
- Import several Excel files from D365 that lists sales orders with Load IDs and consolidate them in one dataframe
- Ask the user how many sessions this should be split into
- Split the dataframe into n x csv files
- Convert the CSV files to Excel

### Clean up the xml_prep folder

In [16]:
path = "data/rel_to_wh/outbound_to_EA/"
dir_list = os.listdir(path)

for i in range(0,len(dir_list)):
    xml_file = path+dir_list[i]
    os.remove(xml_file)

### Import files from D365 containing sales order lines with Load IDs

In [17]:
path = "data/rel_to_wh/inbound_from_D365/"
dir_list = os.listdir(path)

In [18]:
loadfiles_concat = pd.read_excel(path+dir_list[0])
try:
    loadfiles_concat.rename(columns={'Load ID': 'LoadID'}, inplace=True)
except:
    pass

In [19]:
for i in range(1,len(dir_list)):
    xml_file = path+dir_list[i]
    print(str(i)+xml_file)
    temp = pd.read_excel(path+dir_list[i])
    try:
        temp.rename(columns={'Load ID': 'LoadID'}, inplace=True)
    except:
        pass
    loadfiles_concat = pd.concat([loadfiles_concat, temp], ignore_index=True)

1data/rel_to_wh/inbound_from_D365/ZA1-000000747.xlsx
2data/rel_to_wh/inbound_from_D365/ZA1-000000744.xlsx
3data/rel_to_wh/inbound_from_D365/ZA1-000000746.xlsx
4data/rel_to_wh/inbound_from_D365/ZA1-000000745.xlsx
5data/rel_to_wh/inbound_from_D365/ZA1-000000739.xlsx
6data/rel_to_wh/inbound_from_D365/ZA1-000000743.xlsx
7data/rel_to_wh/inbound_from_D365/ZA1-000000740.xlsx
8data/rel_to_wh/inbound_from_D365/ZA1-000000738.xlsx
9data/rel_to_wh/inbound_from_D365/ZA1-000000741.xlsx


In [20]:
loads_to_wh = loadfiles_concat[['LoadID','Description']].copy()
loads_to_wh.drop(columns={'Description'}, inplace=True, axis=1)

In [21]:
loads_to_wh.drop_duplicates(keep='first',inplace=True)
loads_to_wh = loads_to_wh.dropna()
loads_to_wh.reset_index(drop=True, inplace=True)

In [22]:
loads_to_wh['LoadID'] = loads_to_wh['LoadID'].astype(int)
loads_to_wh['LoadID'] = loads_to_wh['LoadID'].astype(str)

In [23]:
loads_to_wh

,LoadID
0,2200010876
1,2200010877
2,2200010878
3,2200010879
4,2200010880
...,...
132,2200010871
133,2200010872
134,2200010873
135,2200010874


### Split the data among the number of user sessions

In [24]:
files_str = input('Enter the number of files into which the loads must be split')
files = int(files_str)

In [25]:
for file in range(1,files+1):
    with open('data/rel_to_wh/outbound_to_EA/wh' + str(file) + '.csv', 'a') as fw:        
        fw.write('LoadID'+'\n')

file = 0
for i in range(len(loads_to_wh)):
    if file <= 27:
        file = file + 1
    else:
        file = 1
    with open('data/rel_to_wh/outbound_to_EA/wh' + str(file) + '.csv', 'a') as fw:        
        fw.write(str(loads_to_wh.loc[i, "LoadID"])+'\n')

### Now convert the CSV files into Excel

In [26]:
path = "data/rel_to_wh/outbound_to_EA/"
dir_list = os.listdir(path)

In [27]:
# 21 user - Release to warehouse , picking and despatch (Roadnet loads).xlsx

for i in range(0,len(dir_list)):
    excel_file = path+str(i+1)+' user - Release to warehouse , picking and despatch (Roadnet loads).xlsx'
    #print(str(i)+xml_file)
    temp = pd.read_csv(path+dir_list[i])
    temp.to_excel(excel_file, index=False)
    os.remove(path+dir_list[i])

In [28]:
raise SystemExit("Stop right here!")

SystemExit: Stop right here!

/home/wo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
